# Week 10: 자율주행 인식 시스템 (Complete)

## 목차
1. 환경 설정 및 설치
2. 차선 인식 (Tier 1-3 비교)
3. 객체 탐지 (YOLOv8)
4. 객체 추적 (ByteTrack)
5. 거리 추정 (IPM)
6. 통합 시스템 (완전판)
7. 성능 벤치마크

---

## 1. 환경 설정 및 설치

In [ ]:
# 필요한 패키지 설치
!pip install -q opencv-python ultralytics numpy matplotlib

# ByteTrack (선택적)
!pip install -q git+https://github.com/ifzhang/ByteTrack.git

print("✅ 설치 완료!")

In [ ]:
# 라이브러리 임포트
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import time

print("✅ 임포트 완료!")

In [ ]:
# 샘플 영상 다운로드
!wget -q https://hsackr-my.sharepoint.com/your-road-video.mp4 -O road_video.mp4
# 또는 본인의 영상 업로드
from google.colab import files
# uploaded = files.upload()  # 파일 업로드

print("✅ 영상 준비 완료!")

## 2. 차선 인식 (Tier 1-3 비교)

### Tier 1: Hough Transform (직선 차선)

In [ ]:
def detect_lanes_hough(frame):
    """Tier 1: Hough Transform 차선 인식"""
    
    # 1. 전처리
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 2. Canny Edge
    edges = cv2.Canny(blur, 50, 150)
    
    # 3. ROI 설정
    height, width = frame.shape[:2]
    roi_vertices = np.array([[
        (0, height),
        (width//2, height//2),
        (width, height)
    ]], dtype=np.int32)
    
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, roi_vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)
    
    # 4. Hough Transform
    lines = cv2.HoughLinesP(
        masked_edges,
        rho=2,
        theta=np.pi/180,
        threshold=50,
        minLineLength=40,
        maxLineGap=100
    )
    
    # 5. 시각화
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
    result = cv2.addWeighted(frame, 0.8, line_image, 1, 0)
    return result


# 테스트
cap = cv2.VideoCapture('road_video.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    result = detect_lanes_hough(frame)
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title('Tier 1: Hough Transform')
    plt.axis('off')
    plt.show()
else:
    print("❌ 프레임 읽기 실패")

### Tier 2: Polynomial Fitting (곡선 차선)

In [ ]:
def detect_lanes_polynomial(frame):
    """Tier 2: Polynomial Fitting 차선 인식"""
    
    # 1. 전처리
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    
    # 2. Perspective Transform
    height, width = frame.shape[:2]
    src = np.float32([
        [width * 0.2, height],
        [width * 0.45, height * 0.6],
        [width * 0.55, height * 0.6],
        [width * 0.8, height]
    ])
    dst = np.float32([
        [width * 0.2, height],
        [width * 0.2, 0],
        [width * 0.8, 0],
        [width * 0.8, height]
    ])
    
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(edges, M, (width, height))
    
    # 3. Histogram
    histogram = np.sum(warped[height//2:, :], axis=0)
    midpoint = len(histogram) // 2
    left_base = np.argmax(histogram[:midpoint])
    right_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # 4. Sliding Window (간략화)
    nwindows = 9
    window_height = height // nwindows
    margin = 100
    minpix = 50
    
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_current = left_base
    right_current = right_base
    
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        win_y_low = height - (window + 1) * window_height
        win_y_high = height - window * window_height
        
        win_xleft_low = left_current - margin
        win_xleft_high = left_current + margin
        win_xright_low = right_current - margin
        win_xright_high = right_current + margin
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            left_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            right_current = int(np.mean(nonzerox[good_right_inds]))
    
    # 5. Polynomial Fitting
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if len(leftx) > 0 and len(rightx) > 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        ploty = np.linspace(0, height-1, height)
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        # 6. 시각화
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        
        cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
        
        Minv = cv2.getPerspectiveTransform(dst, src)
        newwarp = cv2.warpPerspective(color_warp, Minv, (width, height))
        
        result = cv2.addWeighted(frame, 1, newwarp, 0.3, 0)
        return result
    
    return frame


# 테스트
cap = cv2.VideoCapture('road_video.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    result = detect_lanes_polynomial(frame)
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title('Tier 2: Polynomial Fitting')
    plt.axis('off')
    plt.show()

### Tier 1 vs Tier 2 비교

In [ ]:
# FPS 비교
cap = cv2.VideoCapture('road_video.mp4')

# Tier 1 FPS
start = time.time()
count = 0
while count < 100:
    ret, frame = cap.read()
    if not ret:
        break
    _ = detect_lanes_hough(frame)
    count += 1
tier1_time = time.time() - start
tier1_fps = 100 / tier1_time

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # 처음으로

# Tier 2 FPS
start = time.time()
count = 0
while count < 100:
    ret, frame = cap.read()
    if not ret:
        break
    _ = detect_lanes_polynomial(frame)
    count += 1
tier2_time = time.time() - start
tier2_fps = 100 / tier2_time

cap.release()

print(f"Tier 1 (Hough): {tier1_fps:.1f} FPS")
print(f"Tier 2 (Polynomial): {tier2_fps:.1f} FPS")
print(f"속도 차이: {tier1_fps/tier2_fps:.1f}배")

## 3. 객체 탐지 (YOLOv8)

In [ ]:
# YOLOv8 모델 다운로드
from ultralytics import YOLO

# YOLOv8m 모델 (중간 크기)
model = YOLO('yolov8m.pt')

print("✅ YOLOv8 모델 로드 완료!")

In [ ]:
# YOLOv8 테스트
cap = cv2.VideoCapture('road_video.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    # 추론
    results = model(frame, conf=0.3)
    
    # 시각화
    annotated = results[0].plot()
    
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.title('YOLOv8 Object Detection')
    plt.axis('off')
    plt.show()
    
    # 탐지된 객체 출력
    print("\n탐지된 객체:")
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names[cls]
            print(f"  - {label}: {conf:.2f}")

## 4. 객체 추적 (ByteTrack) - 간단 버전

In [ ]:
# 간단한 추적 (ByteTrack 없이 IoU 기반)
class SimpleTracker:
    def __init__(self):
        self.tracks = {}
        self.next_id = 1
    
    def iou(self, box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0
    
    def update(self, detections):
        if len(self.tracks) == 0:
            for det in detections:
                self.tracks[self.next_id] = det
                self.next_id += 1
            return self.tracks
        
        # 매칭
        new_tracks = {}
        used_dets = set()
        
        for track_id, track_box in self.tracks.items():
            best_iou = 0
            best_det_idx = -1
            
            for i, det in enumerate(detections):
                if i in used_dets:
                    continue
                iou_score = self.iou(track_box, det)
                if iou_score > best_iou:
                    best_iou = iou_score
                    best_det_idx = i
            
            if best_iou > 0.3:
                new_tracks[track_id] = detections[best_det_idx]
                used_dets.add(best_det_idx)
        
        # 새 객체
        for i, det in enumerate(detections):
            if i not in used_dets:
                new_tracks[self.next_id] = det
                self.next_id += 1
        
        self.tracks = new_tracks
        return self.tracks


# 테스트
tracker = SimpleTracker()
cap = cv2.VideoCapture('road_video.mp4')

for i in range(10):  # 10프레임만
    ret, frame = cap.read()
    if not ret:
        break
    
    # YOLOv8 탐지
    results = model(frame, verbose=False)
    
    # 박스 추출
    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            detections.append([x1, y1, x2, y2])
    
    # 추적 업데이트
    tracks = tracker.update(detections)
    
    # 시각화
    for track_id, bbox in tracks.items():
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

cap.release()

# 마지막 프레임 표시
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title('Simple Tracking')
plt.axis('off')
plt.show()

## 5. 거리 추정 (IPM)

In [ ]:
def estimate_distance(bbox_bottom_y, image_height=720):
    """간단한 거리 추정"""
    horizon_y = image_height * 0.5
    
    if bbox_bottom_y <= horizon_y:
        return float('inf')
    
    distance = (image_height - horizon_y) / (bbox_bottom_y - horizon_y)
    distance *= 50  # 스케일 (캘리브레이션 필요)
    
    return min(distance, 100)


# 테스트
cap = cv2.VideoCapture('road_video.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    height, width = frame.shape[:2]
    results = model(frame, verbose=False)
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            label = model.names[cls]
            
            # 거리 추정
            distance = estimate_distance(y2, height)
            
            # 색상 (거리별)
            if distance < 10:
                color = (0, 0, 255)  # 빨강
            elif distance < 20:
                color = (0, 165, 255)  # 주황
            else:
                color = (0, 255, 0)  # 초록
            
            # 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {distance:.1f}m", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('Distance Estimation')
    plt.axis('off')
    plt.show()

## 6. 통합 시스템 (완전판)

In [ ]:
# 전체 통합 시스템
class AutonomousDrivingSystem:
    def __init__(self, model_path='yolov8m.pt'):
        self.model = YOLO(model_path)
        self.tracker = SimpleTracker()
        self.track_history = {}
    
    def process_frame(self, frame):
        height, width = frame.shape[:2]
        
        # 1. 차선 인식
        lane_frame = detect_lanes_hough(frame.copy())
        
        # 2. 객체 탐지
        results = self.model(frame, verbose=False)
        
        detections = []
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                detections.append([x1, y1, x2, y2, cls, conf])
        
        # 3. 추적
        det_boxes = [[d[0], d[1], d[2], d[3]] for d in detections]
        tracks = self.tracker.update(det_boxes)
        
        # 4. 시각화
        output = lane_frame
        
        for track_id, bbox in tracks.items():
            x1, y1, x2, y2 = map(int, bbox)
            
            # 거리 추정
            distance = estimate_distance(y2, height)
            
            # 위험도
            if distance < 10:
                color = (0, 0, 255)
                risk = "HIGH"
            elif distance < 20:
                color = (0, 165, 255)
                risk = "MEDIUM"
            else:
                color = (0, 255, 0)
                risk = "LOW"
            
            # 그리기
            cv2.rectangle(output, (x1, y1), (x2, y2), color, 2)
            label = f"ID:{track_id} {distance:.1f}m {risk}"
            cv2.putText(output, label, (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        return output


# 실행
system = AutonomousDrivingSystem('yolov8m.pt')

cap = cv2.VideoCapture('road_video.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    result = system.process_frame(frame)
    
    plt.figure(figsize=(14, 10))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title('Integrated Autonomous Driving System')
    plt.axis('off')
    plt.show()

## 7. 성능 벤치마크

In [ ]:
# 전체 시스템 FPS 측정
cap = cv2.VideoCapture('road_video.mp4')

start = time.time()
count = 0

while count < 50:
    ret, frame = cap.read()
    if not ret:
        break
    _ = system.process_frame(frame)
    count += 1

total_time = time.time() - start
fps = 50 / total_time

cap.release()

print(f"\n🚀 성능 벤치마크 (Google Colab T4 GPU)")
print(f"  - 총 처리 시간: {total_time:.2f}초")
print(f"  - FPS: {fps:.1f}")
print(f"  - 프레임당 시간: {1000/fps:.1f}ms")
print(f"\n💡 실시간 처리 기준: 30 FPS (33.3ms/frame)")

if fps >= 30:
    print("  ✅ 실시간 처리 가능!")
else:
    print("  ⚠️ 최적화 필요")

## 8. 과제 및 다음 단계

### 과제
1. **Tier 3 딥러닝 차선 인식**: HuggingFace Segformer 적용
2. **ByteTrack 통합**: 공식 ByteTrack 라이브러리 사용
3. **카메라 캘리브레이션**: 체스보드로 정확한 거리 계산
4. **TensorRT 최적화**: YOLOv8 → TensorRT 변환
5. **BEV 시각화**: Bird's Eye View 변환 구현

### 참고 자료
- [YOLOv8 공식 문서](https://docs.ultralytics.com/)
- [ByteTrack GitHub](https://github.com/ifzhang/ByteTrack)
- [TuSimple Lane Detection Dataset](https://github.com/TuSimple/tusimple-benchmark)
- [Week 10 강의 자료](lecture_slides.md)

---

**완성!** 🎉

이 노트북은 자율주행 인식 시스템의 핵심 기능을 모두 포함합니다.
실제 프로젝트에 적용하여 더 발전시켜 보세요!